In [ ]:
import numpy as np

def fmridesign(frametimes, slicetimes=None, events=None, S=None, hrf_parameters=None, shift=None):
    # Defaults
    if slicetimes is None:
        slicetimes = np.array([0])
    if events is None:
        events = np.array([[1, 0]])
    if S is None:
        S = np.array([])
    if hrf_parameters is None:
        hrf_parameters = np.array([[5.4, 5.2, 10.8, 7.35, 0.35]])
    if shift is None:
        if isinstance(hrf_parameters, dict):
            shift = np.array([[-4.5, 4.5]])
        else:
            shift = np.array([[-4.5, 4.5]]) * max(max(hrf_parameters[:, 1]) / 5.2, 1)

    n = len(frametimes)
    numslices = len(slicetimes)

    # Keep time points that are not excluded:
    if events.size > 0:
        numevents = events.shape[0]
        eventid = events[:, 0]
        numeventypes = int(np.max(eventid))
        eventime = events[:, 1]
        duration = events[:, 2] if events.shape[1] >= 3 else np.zeros(numevents)
        height = events[:, 3] if events.shape[1] >= 4 else np.ones(numevents)
        mineventime = np.min(eventime)
        maxeventime = np.max(eventime + duration)
    else:
        numeventypes = 0
        mineventime = np.inf
        maxeventime = -np.inf

    numcolS = S.shape[1] if S.size > 0 else 0

    # Set up response matrix:
    dt = 0.02
    startime = min(mineventime, min(frametimes) + min(slicetimes))
    finishtime = max(maxeventime, max(frametimes) + max(slicetimes))
    numtimes = int(np.ceil((finishtime - startime) / dt)) + 1
    numresponses = numeventypes + numcolS
    response = np.zeros((numtimes, numresponses))

    # Fill response matrix based on events
    if events.size > 0:
        height = height / (1 + (duration == 0) * (dt - 1))
        for k in range(numevents):
            type_ = int(eventid[k])
            n1 = int(np.ceil((eventime[k] - startime) / dt)) + 1
            n2 = int(np.ceil((eventime[k] + duration[k] - startime) / dt)) + (duration[k] == 0)
            if n2 >= n1:
                response[n1:n2, type_] += height[k] * np.ones(n2 - n1 + 1)
    
    # Fill response matrix based on S
    if S.size > 0:
        for j in range(numcolS):
            for i in np.nonzero(S[:, j])[0]:
                n1 = int(np.ceil((frametimes[i] - startime) / dt)) + 1
                n2 = int(np.ceil((frametimes[i + 1] - startime) / dt)) if i < n-1 else numtimes
                if n2 >= n1:
                    response[n1:n2, numeventypes + j] += S[i, j] * np.ones(n2 - n1 + 1)

    # Process hrf_parameters and shift
    if isinstance(hrf_parameters, dict):
        if hrf_parameters['T'].shape[0] == 1:
            hrf_parameters['T'] = np.tile(hrf_parameters['T'], (numresponses, 1))
        if hrf_parameters['H'].shape[0] == 1:
            hrf_parameters['H'] = np.tile(hrf_parameters['H'], (numresponses, 1))
    else:
        if hrf_parameters.shape[0] == 1:
            hrf_parameters = np.tile(hrf_parameters, (numresponses, 1))
    
    if shift.shape[0] == 1:
        shift = np.tile(shift, (numresponses, 1))

    eventmatrix = np.zeros((numtimes, numresponses, 4))
    nd = 41
    X_cache_W = np.zeros((nd, numresponses, 5))

    for k in range(numresponses):
        Delta1 = shift[k, 0]
        Delta2 = shift[k, 1]
        
        if isinstance(hrf_parameters, dict):
            numlags = int(np.ceil((np.max(hrf_parameters['T'][k, :]) + Delta2 - Delta1) / dt)) + 1
        else:
            peak1 = hrf_parameters[k, 0]
            fwhm1 = hrf_parameters[k, 1]
            peak2 = hrf_parameters[k, 2]
            fwhm2 = hrf_parameters[k, 3]
            dip = hrf_parameters[k, 4]
            numlags = int(np.ceil((max(peak1 + 3 * fwhm1, peak2 + 3 * fwhm2) + Delta2 - Delta1) / dt)) + 1
        
        numlags = min(numlags, numtimes)
        time = np.arange(numlags) * dt

        # Compute hrfs
        if isinstance(hrf_parameters, dict):
            hrf = np.interp(time, hrf_parameters['T'][k, :], hrf_parameters['H'][k, :], left=0, right=0)
            d_hrf = -np.gradient(hrf, dt)
        else:
            # Implement computation of gamma functions as in MATLAB code
            hrf = np.zeros_like(time)  # Placeholder for the actual hrf computation
            d_hrf = np.zeros_like(time)  # Placeholder for derivative of hrf
        
        # Further processing to fill eventmatrix and X_cache.W...
        # (You'll need to translate the remaining MATLAB calculations for `hrf`, `d_hrf`, and the rest)

    X_cache_X = np.zeros((n, numresponses, 4, numslices))

    for slice_ in range(numslices):
        subtime = int(np.ceil((frametimes + slicetimes[slice_] - startime) / dt)) + 1
        X_cache_X[:, :, :, slice_] = eventmatrix[subtime, :, :]

    TR = (np.max(frametimes) - np.min(frametimes)) / (len(frametimes) - 1)

    X_cache = {
        'TR': TR,
        'X': X_cache_X,
        'W': X_cache_W
    }

    return X_cache